# MS 141 Lecture 10
# Root Finding, Optimization and Non-linear Systems

## Read: Chapter 6 of Newman's book (pages 263-285).

In [1]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (12,6)

$$ \newcommand{\bx}{{\mathbf{x}}} $$

## 1. Root finding
Root finding refers to the general problem of looking for a solution to an equation  $f(x)=0$, namely finding the zeroes (or roots) of a function $f(x)$.<br> 
It is a very general problem with broad applications in scientific computing. For example, if we want to optimize a function  $f(x)$, we need to find its minimum or maximum and therefore solve the equation  $f'(x)=0$. Therefore, *optimization* $-$ finding maxima and minima $-$ corresponds to finding the zeros of the first derivative (or gradient in multiple dimensions). We will first focus on functions of one variable. 

While in some cases an exact solutions to $f(x)=0$ exists, typically one has to find approximate solutions numerically. 
The quadratic formula

$$ x_{\pm} = \frac{-b \pm \sqrt{b^2 - 4 a c}}{2a}$$

gives an exact method for finding the roots of the equation
$$f(x) = a x^2 + b x + c = 0$$ 

There is a general formula to solve a cubic equation and even a quartic (degree 4) equation, but there is no formula for the zeros of a degree 5 or higher polynomial (Abel-Ruffini theorem). There are many more examples of equations with no known analytic solution. 

Our strategy will be to use numerical methods to find approximate solutions. We will examine three - the Bisection, Secant, and Newton methods.

## 2. Bisection method

The simplest root finding algorithm is the __[bisection method](https://en.wikipedia.org/wiki/Bisection_method)__. The algorithm applies to any continuous function $f(x)$ on an interval $[a,b]$ where the value of the function $f(x)$ changes sign from $a$ to $b$. The idea is simple: divide the interval into two parts; since a solution must exist within one of the two subintervals, select the subinterval where the sign of $f(x)$ changes, and then repeat.<br>

The main advantage of the bisection method is its simplicity. The main disadvantages are that it converges slowly because it does not use information about the function (such as its derivative), and it cannot be generalized to more than one dimension.

The bisection algorithm finds roots as follows:
1. Choose a starting interval $[a_0,b_0]$ such that $f(a_0)f(b_0)<0$ ( $f$ changes sign in the interval).
2. Compute $f(m_0)$ where $m_0 = \frac{a_0 + b_0}{2}$ is the midpoint.
3. Determine the next subinterval $[a_1,b_1]$:<br>
    a. If $f(a_0)f(m_0)<0$, then set the next interval between $a_1 = a_0$ and $b_1 = m_0$.<br>
    b. If $f(b_0)f(m_0)<0$, then set the next interval between $a_1 = m_0$ and $b_1 = b_0$.<br>
4. Repeat (2) and (3) for a given number of cycles $N$, after which the interval reaches a length of $\frac{b-a}{2^N}$. 
5. Return the midpoint value as the solution. Its error relative to the exact solution will be of order $\frac{b-a}{2^{N+1}}$.<br>
If one selects a small threshold $\varepsilon$ for the solution, such that $\varepsilon = \frac{b-a}{2^{N}}$, this threshold is reached in roughly $N = \log(\frac{b-a}{\varepsilon})$ steps.
(Image source: Wiki)
<img src="images/Bisection_method.png" style="width: 300px;"/>

In [2]:
#Root finding using the bisection method 
'''
a,b: interval limits
f: function whose roots we are searching 
N: number of iterations 
Returns: midpoint of the Nth interval (approximate root of f)
'''

def bisection(f,a,b,N): #alternatively, could pass a threshold
    if f(a)*f(b) >= 0:
        print("Bisection method fails.")
        return None
    a_n = a
    b_n = b
    for n in range(1,N+1):
        # midpoint
        m_n = (a_n + b_n)/2
        
        # zero is left of midpoint
        if f(a_n)*f(m_n) < 0:
            a_n = a_n
            b_n = m_n
            
        # zero is right of midpoint    
        elif f(b_n)*f(m_n) < 0:
            a_n = m_n
            b_n = b_n
            
        elif f(m_n) == 0:
            print("Found exact solution.")
            return m_n
        
        else:
            print("Bisection method fails.")
            return None
        
    return (a_n + b_n)/2

In [4]:
# Solve x^2 - x - 1 = 0
f = lambda x: x**2 - x - 1
approx_phi = bisection(f,1,2,50) #50
print(approx_phi)

# Check solution
print (f(approx_phi))

Found exact solution.
1.618033988749895
0.0


In [5]:
# Solve (2x-1)(x-3) = 0
f = lambda x: (2*x - 1)*(x - 3)
bisection(f,0,1,10)

Found exact solution.


0.5

### Bisection method with SciPy

SciPy's `optimize.bisect` implements the bisection method for a user-defined number of steps.

In [6]:
from scipy import optimize
f = lambda x: x**2 - x - 1
optimize.bisect(f, 1, 2, maxiter=100)

1.6180339887487207

In [7]:
f = lambda x: (2*x - 1)*(x - 3)
optimize.bisect(f, 0, 1, maxiter=2)

0.5

## 3. Secant method

The __[secant method](https://en.wikipedia.org/wiki/Secant_method)__ is a root-finding algorithm that uses a succession of secant lines to better approximate the root of a function.<br> 
The secant method can be thought of as a finite-difference approximation of Newton's method (see below).<br> 

However, the method was developed independently of Newton's method and predates it by over 3,000 years.<br> 
It relies on the simple assumption that near a root a function is usually linear.

Consider the line connecting the endpoint values $(x_0,f(x_0))$ and $(x_1,f(x_1))$ of a function $f(x)$.<br> 
The line connecting these two points is called the secant line and is given by the formula 

$$ y(x) = \frac{f(x_1) - f(x_0)}{x_1-x_0} (x-x_1) + f(x_1)$$

The secant crosses the $x$ axis when $y=0$, and thus at the point $x=x_2$:

$$ x_2 = x_1 - f(x_1) \frac{x_1-x_0}{f(x_1)-f(x_0)}.$$

The next secant is built between the points $(x_1, f(x_1))$ and $(x_2, f(x_2))$, and we call $x_3$ its intersection with the $x$ axis. We continue this process, solving for $x_3$, $x_4$, etc., until we reach a sufficient precision $-$ that is, a sufficiently small difference between $x_n$ and $x_{n−1}$.

The secant algorithm can be summarized as follows:<br>
1. Choose a starting interval $[x_0,x_1]$ such that $f(x_0)f(x_1)<0$ ( $f$ changes sign in the interval).
2. Find $x_2 = x_{1}-f(x_{1}){\frac {x_{1}-x_{0}}{f(x_{1})-f(x_{0})}}$.
3. Repeat the procedure in the next subinterval $[x_1,x_2]$, solving for $x_3$, and so on.<br> 
   At step $n$, the next iterate solution is
   
$$ \boxed{x_{n+1}=x_{n}-f(x_{n})\,{\frac {x_{n}-x_{n-1}}{f(x_{n})-f(x_{n-1})}}}$$

4. Repeat steps 2) and 3) until the interval $[x_N,x_{N+1}]$ reaches a predetermined length. 
5. Return the value $x_N$, the $x$-intercept of the $N^{\rm{th}}$ subinterval. This will be the approximate solution to $f(x)=0$. <br>
(Image source: Wiki)
<img src="images/Secant_method.png" style="width: 300px;"/>

Note that unlike the bisection method, the secant method does not require that the root remain bracketed.<br> 
If the initial values are not close enough to the root, there is no guarantee that the secant method will converge.
When it converges, the secant method converges faster than bisection as it uses knowledge of the function to find its roots.

In [8]:
def secant(f,x0,x1,N):
    x_old = x0 
    x_new = x1 
    
    eps=1.e-10
    for i in range(N):
        if (abs(f(x_new)-f(x_old)) < eps):
            return x_new
        
        # find x_n+1
        x_tmp = x_new - 1.0*f(x_new)*(x_new-x_old)/(f(x_new)-f(x_old))
        
        # debug: print root
        # print (x_tmp)
        
        x_old = x_new
        x_new = x_tmp
        
    return x_new

In [9]:
# Solve x^2 - x - 1 = 0
f = lambda x: x**2 - x - 1
root = secant(f,1,2,25)
print(root)

# Check solution
f(root)

1.6180339887498947


-2.220446049250313e-16

### Secant method with SciPy

The `optimize.newton` function uses the secant method if a derivative of f is not provided.

In [10]:
from scipy import optimize
f = lambda x: x**2 - x - 1
# use x_0=1.3
root = optimize.newton(f, 1.3)
print(root)

1.6180339887498947


## 4. Newton method

The [Newton method](https://en.wikipedia.org/wiki/Newton%27s_method) (also known as Newton-Raphson method) is a widely used root finding approach. It uses a single point as the initial guess of the root of a function, and using the direction of the tangent it finds successively better approximations to the root. It requires the *computation of the derivative*.<br> 
It can be implemented in one or multiple dimensions. We will first investigate its implementation in one dimension.<br>

The idea of the Newton method is that if an initial guess point $x_0$ is near a solution of $f(x)=0$, then we can approximate $f(x)$ by the tangent line at $x_0$ <br> 
and compute the $x$-intercept of the tangent line. The equation of the tangent line at $x_0$ is<br>

$$ y (x) = f'(x_0) (x-x_0) + f(x_0)$$

The $x$-intercept is the solution $x_1$ of the equation $y=0$, which gives<br>

$$ x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}$$

Repeating this procedure, we obtain a zero of $f$ from sequence of values $x_n$ given by the recursive formula

$$\boxed {x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)},}$$

through which one progressively finds better approximations of the solution to $f(x)=0$.
When it converges, Newton's method usually converges quickly and this is its main advantage. However, Newton's method is not guaranteed to converge and this is obviously a disadvantage, especially compared to the bisection method, which is guaranteed to converge to a solution (provided it starts with an interval containing a root). 

Newton's method requires computing values of the derivative of the function. This is potentially a disadvantage if the derivative is difficult to compute (e.g., in multiple dimensions). [Newton's method for optimization](https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization) is applied to the derivative $f'(x)$ of a twice-differentiable function $f(x)$ to find the roots of the derivative (the solutions to $f'(x) = 0)$. For optimization, the maxima / minima of $f(x)$ are obtained from the sequence: 

$$\boxed{ x_{n+1} = x_n - \frac{f'(x_n)}{f''(x_n)}, }$$

which involves computing the second derivatives of $f$.

(Image source: Wiki)
<img src="images/Newton_method.gif" style="width: 500px;"/>

Here is an example of Newton's method. We need to provide the derivative $f'(x)$ as input or as a callable function. In our case, we will compute the derivative analytically, but in general a numerical approximation of $f'(x)$ is needed. The algorithm stops either when the function is less than a small threshold value and thus close enough to zero,  $f(x_n) < \varepsilon$, or after a maximum number of iterations. Also, if $f'(x_n)=0$, the algorithm fails and needs to be stopped.

In [11]:
def newton(f,Df,x0,epsilon,max_iter):
    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function in f(x)=0.
    Df : derivative function of f(x).
    x0 : number
        Initial guess for a solution f(x)=0.
    epsilon : number
        Stopping criterion abs(f(x)) < epsilon.
    max_iter : integer
        Maximum number of iterations of Newton's method.

    Returns
    -------
    xn : number
    '''
    xn = x0
    for n in range(0,max_iter):
        
        fxn = f(xn)
        
        if abs(fxn) < epsilon:
            print('Found solution after',n,'iterations.')
            return xn
        
        if Df(xn) == 0:
            print('Zero derivative. No solution found.')
            return None
        
        xn = xn - fxn/Df(xn)
        
    print('No solution found in ', max_iter, ' iterations.')
    return None

In [12]:
f = lambda x: x**2 - x - 1
Df = lambda x: 2*x - 1
root = newton(f,Df,1,1e-12,10)
print(root)
#check
print (f(root))

Found solution after 5 iterations.
1.618033988749989
2.1049828546892968e-13


In [13]:
f = lambda x: (2*x - 1)*(x - 3)
Df = lambda x: 2*(x-3) + (2*x - 1)
root = newton(f,Df,0.99,1e-10,10)
print (root)

Found solution after 5 iterations.
0.5


### Newton method with SciPy

In [14]:
from scipy import optimize
f = lambda x: x**2 - x - 1
root = optimize.newton(f, 1.5, fprime=lambda x: 2*x - 1)
print (root)

1.618033988749895


In [15]:
f = lambda x: (2*x - 1)*(x - 3)
root = optimize.newton(f, 1.5, fprime=lambda x: 2*(x-3) + (2*x - 1))
print (root)

0.5


## 5. Newton method in more than one variable

To generalize the Newton method to more than one variable, it is useful to derive it using Taylor expansion:

$$\begin{aligned}
        y &= f(x_n) + (x - x_n) f'(x_n) \\
        y & = 0 \quad \implies \quad x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}.
      \end{aligned}$$

For a function of more than one variable, the points in $N$-dimensional space become $\bx = (x_1,x_2,\ldots,x_N)$ and the derivative $f'(x)$ becomes the gradient $\mathbf{\nabla f} = \frac{\partial f}{\partial \bx}$. To solve $f(\bx)=0$ we rewrite the Taylor expansion as:

$$ \boxed{ \mathbf{\nabla f} \cdot (\bx_{n+1} - \bx_n) = -\, f(\bx_n). }$$

Using this scheme, at each iteration step we can update the point $\bx_n$ by solving one equation in $N$ unknowns. 

**Optimization.** It is far more common to use the multi-variable Newton approach to find local maxima or minima of $f$, which requires the gradient to vanish, $\mathbf{\nabla f} = 0$. Newton's method in one variable uses the first and second derivatives to find maxima or minima. In higher dimensions, Newton's method uses the gradient and the Hessian matrix (matrix of second derivatives) of the function to be minimized. Now the iteration involves the symmetric Hessian matrix:

$$H_{ij} = \frac{\partial^2 f}{\partial x_i \partial x_j}.$$ 

At each step, Newton's scheme becomes a systems of $N$ linear equations in $N$ unknowns:

$$ H (\bx_{n+1} - \bx_n) = -\, \mathbf{\nabla f}(\bx_n). $$

Therefore, each step in Newton's algorithm requires updating the Hessian matrix plus $\mathcal{O}(N^3)$ operations to solve a linear system.

Computing the Hessian matrix at each step and solving a linear system is computationally costly. [*Quasi-Newton methods*](https://en.wikipedia.org/wiki/Quasi-Newton_method), such as the Broyden and BFGS methods, are widely used optimization schemes in which the Hessian matrix is not calculated explicitly, but rather only estimated approximately, for example using finite differences as in the secant approach. The iterative scheme becomes:

$$\bx_{n+1} - \bx_n = -\alpha_n B_n^{-1}\, \mathbf{\nabla f}(\bx_n),$$

where $\alpha_n$ is a line search parameter and $B_n$ an approximate Hessian. The gradient descent method we saw earlier can also be regarded as a quasi-Newton scheme with $B_n = I$ and appropriate $\alpha_n$. 
Quasi-Newton methods are typically more robust than the pure Newton method itself, require no second derivative evaluation, and typically cost only $\mathcal{O}(N^2)$ per iteration step. For this reason they are widely used in practice, particularly the [BFGS method](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm), which is available in SciPy as the `scipy.optimize.fmin_bfgs` function. 

**Nonlinear systems.** Another generalization of the Newton method can be used to solve system of nonlinear equations.<br> The nonlinear system with $N$ equations and $N$ unknowns can be written as:

$$
\begin{aligned}
f_1\,(x_1,x_2, \ldots, x_N) &= 0 \\
f_2\,(x_1,x_2, \ldots, x_N) &= 0 \\
\vdots \\
f_N\,(x_1,x_2, \ldots, x_N) &= 0
\end{aligned}
$$

or in more compact form, $\mathbf{f}(\bx)=0$. In this case, the gradient is generalized to the Jacobian:

$$ J_{ij}(\bx) = \frac{\partial f_i}{\partial x_j} $$

Newton's method becomes the matrix equation

$$J (\bx_n) \cdot ( \bx_{n+1} - \bx_n ) = -\, {\boldsymbol{f}}(\bx_n)$$

which requires to solve a linear system at each step. In the algorithm, one starts with an initial guess $\bx_0$ and computes $\mathbf{f}(\bx_0)$. At step $n$:

1.  Compute ${\boldsymbol{f}}(\boldsymbol{x}_n)$.

2.  Compute the Jacobian $J (\boldsymbol{x}_n)$.

3.  Solve for the search direction ${\boldsymbol{s}} = \bx_{n+1} - \bx_n$ from
    $J (\boldsymbol{x}_n) {\boldsymbol{s}} = -{\boldsymbol{f}}(\boldsymbol{x}_n)$.

4.  Compute $\boldsymbol{x}_{n+1} = \boldsymbol{x}_n + {\boldsymbol{s}}$.

Note that the Jacobian is never inverted explicitly.

###  Nonlinear system example

We illustrate Newton's method by solving the nonlinear system

$$\begin{aligned}
        x_1 + 2\,x_2 - 2 & = 0 \\
        x_1^2 + 4\,x_2^2 - 4 & = 0.
      \end{aligned}$$

with Jacobian matrix

$$ J =  \begin{pmatrix}
      1 & 2 \\
      2\,x_1 & 8\,x_2
    \end{pmatrix}
$$    

We take $\bx_0 = (1\,\,\,2)^T$ as the initial guess, and attempt to find the exact solution $\bx^* = (0\,\,\,1)^T$.

In [16]:
import numpy as np

def newton_system(f, df, x0, nsteps=10):
    
    N = len(x0)
    
    x = np.zeros((N,nsteps+1))
    x[:, 0] = x0
    
    for n in range(nsteps):
        fx = f(x[:,n])
        J = df(x[:,n])
        
        # J(x_n+1 - x_n) = - f(x_n)
        dx = np.linalg.solve(J, -fx) 
        
        x[:, n+1] = x[:,n] + dx
    return x

In [17]:
def f(x):
    f = np.zeros_like(x)
    f[0] = x[0] + 2*x[1] - 2.0
    f[1] = x[0]**2 + 4.0*x[1]**2 - 4.0
    return f

def df(x):
    df = np.zeros((len(x),len(x)))
    df[0,0] = 1.0
    df[0,1] = 2.0
    df[1,0] = 2.*x[0]
    df[1,1] = 8.*x[1]
    return df

nsteps = 10
x0 = np.array((1.0,2.0))
result = newton_system(f, df, x0, nsteps)

print ('Iterative results from Newton:\n')

for i in range (nsteps+1):
    np.set_printoptions(precision=10,suppress=True) #format
    print (result[:,i])

Iterative results from Newton:

[1. 2.]
[-0.8333333333  1.4166666667]
[-0.1893939394  1.0946969697]
[-0.0150791353  1.0075395677]
[-0.0001120013  1.0000560006]
[-0.0000000063  1.0000000031]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]


### References

R. Fletcher, Practical Methods of Optimization, 2nd Ed. (Wiley)